Question 1: [Index] S&P 500 Stocks Added to the Index
Which year had the highest number of additions?

Using the list of S&P 500 companies from Wikipedia's S&P 500 companies page, download the data including the year each company was added to the index.

Hint: you can use pandas.read_html to scrape the data into a DataFrame.

Steps:

Create a DataFrame with company tickers, names, and the year they were added.
Extract the year from the addition date and calculate the number of stocks added each year.
Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).
Context:

"Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 (Nasdaq article).

Additional: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [2]:
!pip install pandas

In [ ]:
!pip install lxml

In [2]:
import pandas as pd

# Scrape the S&P 500 companies table from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]  # The first table contains the list of S&P 500 companies

# Display the first few rows
sp500_df.head()


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
sp500_df['Year added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce').dt.year

In [21]:
#quick check 
year_added = sp500_df['Year added']
valid_years = year_added.dropna()
valid_years = valid_years[(valid_years >= 1900) & (valid_years <= 2025)]
if len(valid_years) == len(sp500_df):
    print("data valid")
else:
    print("check your data")


data valid


In [66]:
# Count the number of companies added each year (excluding 1957)
year_counts = sp500_df[sp500_df['Year added'] != 1957]['Year added'].value_counts()

# Get the top years with the most additions
year_counts.nlargest(1)
print("Q: Year had the highest number of addition\nA:", year_counts.nlargest(1).index[0])


Q: Year had the highest number of addition
A: 2017


In [71]:
#add the number of years
sp500_df['Years in sp500'] = abs((datetime.today() - pd.to_datetime(sp500_df['Date added'], errors='coerce')).dt.days) // 365

# Count the number of companies been index for more than 20 years
more_than_20_years = sp500_df[sp500_df['Years in sp500'] > 20]
print("Q: How many current S&P 500 stocks have been in the index for more than 20 years?\nA:", len(more_than_20_years))

Q: How many current S&P 500 stocks have been in the index for more than 20 years?
A: 215
